# NCAR CESM2-LENS on AWS

In [ ]:
if "google.colab" in str(get_ipython()):
    print("Running on CoLab")
    !pip install zarr==2.18 cftime s3fs
else:
    print("Not running on CoLab")

Similar to the CMIP6 dataset on google cloud, the NCAR CESM2-LENS dataset is available on AWS and to access it we only need to use the `s3fs` package. More information about this collection can be found on their [website](https://ncar.github.io/cesm2-le-aws/model_documentation.html#overview).

First we need to do an anonymous log-in to AWS file system:

In [ ]:
import pandas as pd
import s3fs
import xarray as xr

In [ ]:
fs = s3fs.S3FileSystem(anon=True)

Most of the datacenters that host their model output in cloud storages have a .csv file hosted that you can use to filter the data you want.

In [ ]:
df = pd.read_csv(
    "https://ncar-cesm2-lens.s3-us-west-2.amazonaws.com/catalogs/aws-cesm2-le.csv"
)
df.head()

In [ ]:
df.variable.unique()

`pandas` is our best friend here to query the data we want. 

In [ ]:
filter_df = df.query(
    "variable == 'PRECC' and experiment == 'historical' and frequency == 'monthly' and forcing_variant == 'cmip6'"
)
filter_df

Once we have the data we want, we can use the `path` column to the the data location and load it into python

In [ ]:
mapper = fs.get_mapper(filter_df.path.values[0])
xrdata = xr.open_zarr(mapper, consolidated=True)
xrdata